# Task 3

1. Limpieza y Pre-procesamiento

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
#a. Eliminación de Redundancia
# Cargar dataset
df = pd.read_csv("high_diamond_ranked_10min.csv")
y = df['blueWins'] # Variable objetivo

# Eliminar columnas con Data Leakage o redundancia
X = df.drop(columns=['blueWins', 'gameId'])

# Eliminar todas las columnas del lado rojo
X = X.loc[:, ~X.columns.str.startswith('red')]

X.head()

,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,blueTotalExperience,blueTotalMinionsKilled,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin
0,28,2,1,9,6,11,0,0,0,0,17210,6.6,17039,195,36,643,-8,19.5,1721.0
1,12,1,0,5,5,5,0,0,0,0,14712,6.6,16265,174,43,-2908,-1173,17.4,1471.2
2,15,0,0,7,11,4,1,1,0,0,16113,6.4,16221,186,46,-1172,-1033,18.6,1611.3
3,43,1,0,4,5,5,1,0,1,0,15157,7.0,17954,201,55,-1321,-7,20.1,1515.7
4,75,4,0,6,6,6,0,0,0,0,16400,7.0,18543,210,57,-1004,230,21.0,1640.0


In [ ]:
#b. Escalado (Obligatorio para SVM)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

scaler = StandardScaler()

# Ajustar SOLO con train
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar al test
X_test_scaled = scaler.transform(X_test)

'''Si las variables no están escaladas, aquellas con valores más grandes dominan el cálculo del margen, 
afectando negativamente el desempeño del modelo. Ahora, los modelos basados en árboles toman decisiones 
mediante comparaciones y umbrales, no mediante distancias.'''

'Si las variables no están escaladas, aquellas con valores más grandes dominan el cálculo del margen, \nafectando negativamente el desempeño del modelo. El escalado asegura que todas las características contribuyan \nde manera equitativa al modelo, mejorando su rendimiento y capacidad de generalización.'

2. Support Vector Machines:

In [ ]:
#a. Entrene un modelo SVM con Kernel Lineal.
